In [ ]:
!pip install qiskit

In [8]:
# initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit import QuantumCircuit, Aer, execute

# import basic plot tools
from qiskit.visualization import plot_histogram

n = 3 # number of qubits used to represent s
s = '011'   # the hidden binary string

# We need a circuit with n qubits, plus one ancilla qubit
# Also need n classical bits to write the output to
q = QuantumRegister(4)
c = ClassicalRegister(3)

bv_circuit = QuantumCircuit(q, c)

# put ancilla in state |->
bv_circuit.h(q[3])
bv_circuit.z(q[3])

# Apply Hadamard gates before querying the oracle
for i in range(n):
    bv_circuit.h(q[i])
    
# Apply barrier 
bv_circuit.barrier()

# Apply the inner-product oracle
s = s[::-1] # reverse s to fit qiskit's qubit ordering
for t in range(n):
    if s[t] == '0':
        bv_circuit.i(q[t])
    else:
        bv_circuit.cx(q[t],q[n])
        
# Apply barrier 
bv_circuit.barrier()

#Apply Hadamard gates after querying the oracle
for i in range(n):
    bv_circuit.h(q[i])

# Measurement
for i in range(n):
    bv_circuit.measure(i, i)

backend = Aer.get_backend('qasm_simulator')
result = execute(bv_circuit,backend)
out_state = result.result()
print(out_state.get_counts())
bv_circuit.draw()

{'011': 1024}


┌───┐      ░                 ░ ┌───┐┌─┐      
q18_0: ┤ H ├──────░───■─────────────░─┤ H ├┤M├──────
       ├───┤      ░   │             ░ ├───┤└╥┘┌─┐   
q18_1: ┤ H ├──────░───┼─────────■───░─┤ H ├─╫─┤M├───
       ├───┤      ░   │  ┌───┐  │   ░ ├───┤ ║ └╥┘┌─┐
q18_2: ┤ H ├──────░───┼──┤ I ├──┼───░─┤ H ├─╫──╫─┤M├
       ├───┤┌───┐ ░ ┌─┴─┐└───┘┌─┴─┐ ░ └───┘ ║  ║ └╥┘
q18_3: ┤ H ├┤ Z ├─░─┤ X ├─────┤ X ├─░───────╫──╫──╫─
       └───┘└───┘ ░ └───┘     └───┘ ░       ║  ║  ║ 
 c6: 3/═════════════════════════════════════╩══╩══╩═
                                            0  1  2